# Character Issue Fix

Quick fix for special characters that were broken on display. 

In [1]:
import findspark
findspark.init("/home/raven/.spark")

In [2]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [3]:
spark = SparkSession.builder \
   .master("local[4]") \
   .appName("Orizon - Raio X") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
sc = spark.sparkContext
sqlContext= SQLContext(sc, sparkSession=spark)

In [4]:
df = sqlContext.read.format('csv').options(header='true', inferSchema='true', delimiter="\t").load('data_30726034.tsv')

In [5]:
df.printSchema()

root
 |-- categoria': string (nullable = true)
 |-- id_cliente': string (nullable = true)
 |-- cliente': string (nullable = true)
 |-- id_lote': string (nullable = true)
 |-- id_conta': string (nullable = true)
 |-- guia_prestador': string (nullable = true)
 |-- id_item': string (nullable = true)
 |-- id_prestador': string (nullable = true)
 |-- prestador': string (nullable = true)
 |-- cnpj': string (nullable = true)
 |-- uf_prestador': string (nullable = true)
 |-- cidade_prestador': string (nullable = true)
 |-- id_beneficiario': string (nullable = true)
 |-- sexo': string (nullable = true)
 |-- data_nascimento': string (nullable = true)
 |-- status': string (nullable = true)
 |-- cid': string (nullable = true)
 |-- cid2': string (nullable = true)
 |-- cid3': string (nullable = true)
 |-- cid4': string (nullable = true)
 |-- senha': string (nullable = true)
 |-- crm_solicitante': string (nullable = true)
 |-- uf_crm_solicitante': string (nullable = true)
 |-- cbos_solicitante': stri

In [6]:
df.select("descricao_despesa'").head(50)

[Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="ARTROPLASTIA TOTAL DE JOELHO COM IMPLANTES TRATA'"),
 Row(descricao_despesa'="Artrotomia - tratamento cirÃºrgico'"),
 Row(descricao_despesa'="Realinhamentos do aparelho extensor - tratamento cirÃºrgico'"),
 Row(descricao_despesa'="Artroplastia total de joelho com implantes - tratamento cirÃ

Broken character example: Realinhamentos do aparelho extensor - tratamento cirÃºrgico'

In [7]:
def string_encode(string): # function to fix character encoding
    return string.encode("ISO-8859-1").decode("UTF-8")

In [8]:
encode_udf = udf(string_encode, StringType()) 
df = df.withColumn("descricao_despesa'", encode_udf("descricao_despesa'")) # applying function the the entire column

In [9]:
df.select("descricao_despesa'").head(50)

[Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="30726034'"),
 Row(descricao_despesa'="ARTROPLASTIA TOTAL DE JOELHO COM IMPLANTES TRATA'"),
 Row(descricao_despesa'="Artrotomia - tratamento cirúrgico'"),
 Row(descricao_despesa'="Realinhamentos do aparelho extensor - tratamento cirúrgico'"),
 Row(descricao_despesa'="Artroplastia total de joelho com implantes - tratamento cirú'"

As seen above, "Realinhamentos do aparelho extensor - tratamento cirúrgico'" now displays correctly.